# Topog form stress using MOM025 DATA

Recent update 18 Sept 2017

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import os
import pickle
from numpy import math

import pandas as pd
import xarray as xr
from glob import glob
from mpl_toolkits.basemap import Basemap, shiftgrid


# Load Data

In [32]:
## Load data

%time
yy=217

OceanFile1 = '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55_ryf_spinup7/output100/ocean/ocean_grid.nc'
dsx1 = xr.open_dataset(OceanFile1, engine='netcdf4')
kmt = dsx1.kmt.isel(yt_ocean=yy)  # "number of depth levels on t-grid"
depth = -dsx1.ht.isel(yt_ocean=yy)   ## H
dxu = dsx1.dxu.isel(yu_ocean=yy) ## dxu

OceanFile2 =  '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55_ryf_spinup7/output100/ocean/ocean_month.nc'
dsx2 = xr.open_dataset(OceanFile2, engine='netcdf4')
taux = dxu*dsx2.tau_x.isel(yu_ocean=yy).mean('time')  ## tau_x
pbot = dsx2.pbot_t.mean('time').isel(yt_ocean=yy)     ## p_bot
eta0 = dsx2.eta_t.mean('time') ## \eta
eta = dsx2.sea_level.mean('time') ## sea_level

OceanFile3 =  '/g/data3/hh5/tmp/cosima/access-om2-025/025deg_jra55_ryf_spinup7/output100/ocean/ocean.nc'
dsx3 = xr.open_dataset(OceanFile3, engine='netcdf4')
rho = dsx3.rho.mean('time').isel(yt_ocean=yy)        ## \rho in-situ
dz = dsx3.dzt.mean('time').isel(yt_ocean=yy)      ## t-cell thickness
g = 9.8

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.96 µs


In [33]:
dz[30,1391:1411]

<xarray.DataArray 'dzt' (xt_ocean: 20)>
array([ 181.452835,  181.446899,  181.43631 ,  181.427628,  181.431198,
        181.430984,  181.431015,  181.430725,  181.42485 ,  181.41304 ,
        181.394485,   90.983482,   36.275105,  168.066025,  181.397858,
        181.409592,  181.413986,  181.42865 ,  181.441879,  181.449738])
Coordinates:
  * xt_ocean  (xt_ocean) float64 67.88 68.12 68.38 68.62 68.88 69.12 69.38 ...
    yt_ocean  float64 -57.23
    st_ocean  float64 1.498e+03

## Step 1: 
## define water thickness as h: $h = -\int dz$
## define bottom pressure as p: $p = \int \rho*g*dz$

In [34]:
h = -dz.sum('st_ocean')

dp = rho*g*dz
p = dp.sum('st_ocean')

## Step 2:
## Calculate $\sum \Delta p*\Delta z$
## $\Delta p=p(east)-p(west)$

In [35]:
z=np.zeros(kmt.shape).astype(np.int64) 
z[:]=kmt[:].values-1
dp = 0

a=0.025
b=0.025


pb=np.zeros(p.shape)
pb[:]=p[:].values

for x in range(0,1440):
    dkmt1=z[x]-z[x-1]
    if dkmt1==0:
        dz1=dz[z[x],x-1]
        dz2=dz[z[x],x]
        if (dz1-dz2)<0:
            rho1=rho[z[x],x]
            dp=dp+(pb[x]*(dz2-dz1+a)-g*rho1*(dz2-dz1)**2/2)
        elif (dz1-dz2)>0:
            rho1=rho[z[x-1],x-1]
            dp=dp-(pb[x-1]*(dz1-dz2+b)-g*rho1*(dz2-dz1)**2/2)
    elif dkmt1==1:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x] - dz[z[x]-1,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2)-g*rho1*dz1*dz2
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2)
    elif dkmt1==-1:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1] - dz[z[x-1]-1,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2)-g*rho1*dz1*dz2
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2)
    elif dkmt1==2:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]+a
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]- dz[z[x]-2,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2+dz3)
               -g*rho1*dz1*(dz2+dz3)
               -g*rho2*dz2*(dz3)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2)
    elif dkmt1==-2:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]+b
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]- dz[z[x-1]-2,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2+dz3)
               -g*rho1*dz1*(dz2+dz3)
               -g*rho2*dz2*(dz3)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2)       
    elif dkmt1==3:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]+a
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]+a
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]- dz[z[x]-3,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4)
               -g*rho1*dz1*(dz2+dz3+dz4)
               -g*rho2*dz2*(dz3+dz4)
               -g*rho3*dz3*(dz4)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2)        
    elif dkmt1==-3:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]+b
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]+b
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]- dz[z[x-1]-3,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4)
               -g*rho1*dz1*(dz2+dz3+dz4)
               -g*rho2*dz2*(dz3+dz4)
               -g*rho3*dz3*(dz4)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2)
    elif dkmt1==4:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]+a
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]+a
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]+a        
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]- dz[z[x]-4,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5)
               -g*rho2*dz2*(dz3+dz4+dz5)
               -g*rho3*dz3*(dz4+dz5)
               -g*rho4*dz4*(dz5)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2)
    elif dkmt1==-4:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]+b
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]+b
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]+b        
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]- dz[z[x-1]-4,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5)
               -g*rho2*dz2*(dz3+dz4+dz5)
               -g*rho3*dz3*(dz4+dz5)
               -g*rho4*dz4*(dz5)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2)
    elif dkmt1==5:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]+a
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]+a
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]+a    
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]+a
        rho6=rho[z[x]-5,x]
        dz6=dz[z[x]-5,x]- dz[z[x]-5,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5+dz6)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6)
               -g*rho3*dz3*(dz4+dz5+dz6)
               -g*rho4*dz4*(dz5+dz6)
               -g*rho5*dz5*(dz6)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2)         
    elif dkmt1==-5:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]+b
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]+b
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]+b    
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]+b
        rho6=rho[z[x-1]-5,x-1]
        dz6=dz[z[x-1]-5,x-1]- dz[z[x-1]-5,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5+dz6)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6)
               -g*rho3*dz3*(dz4+dz5+dz6)
               -g*rho4*dz4*(dz5+dz6)
               -g*rho5*dz5*(dz6)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2)         
    elif dkmt1==6:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]+a
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]+a
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]+a    
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]+a
        rho6=rho[z[x]-5,x]
        dz6=dz[z[x]-5,x]+a       
        rho7=rho[z[x]-6,x]
        dz7=dz[z[x]-6,x]- dz[z[x]-6,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7)
               -g*rho4*dz4*(dz5+dz6+dz7)
               -g*rho5*dz5*(dz6+dz7)
               -g*rho6*dz6*(dz7)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2)         
    elif dkmt1==-6:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]+b
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]+b
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]+b    
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]+b
        rho6=rho[z[x-1]-5,x-1]
        dz6=dz[z[x-1]-5,x-1]+b        
        rho7=rho[z[x-1]-6,x-1]
        dz7=dz[z[x-1]-6,x-1]- dz[z[x-1]-6,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7)
               -g*rho4*dz4*(dz5+dz6+dz7)
               -g*rho5*dz5*(dz6+dz7)
               -g*rho6*dz6*(dz7)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2)        
    elif dkmt1==7:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]+a
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]+a
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]+a    
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]+a
        rho6=rho[z[x]-5,x]
        dz6=dz[z[x]-5,x]+a  
        rho7=rho[z[x]-6,x]
        dz7=dz[z[x]-6,x]+a         
        rho8=rho[z[x]-7,x]
        dz8=dz[z[x]-7,x]- dz[z[x]-7,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8)
               -g*rho4*dz4*(dz5+dz6+dz7+dz8)
               -g*rho5*dz5*(dz6+dz7+dz8)
               -g*rho6*dz6*(dz7+dz8)
               -g*rho7*dz7*(dz8)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2)                 
    elif dkmt1==-7:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]+b
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]+b
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]+b    
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]+b
        rho6=rho[z[x-1]-5,x-1]
        dz6=dz[z[x-1]-5,x-1]+b
        rho7=rho[z[x-1]-6,x-1]
        dz7=dz[z[x-1]-6,x-1]+b        
        rho8=rho[z[x-1]-7,x-1]
        dz8=dz[z[x-1]-7,x-1]- dz[z[x-1]-7,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8)
               -g*rho4*dz4*(dz5+dz6+dz7+dz8)
               -g*rho5*dz5*(dz6+dz7+dz8)
               -g*rho6*dz6*(dz7+dz8)
               -g*rho7*dz7*(dz8)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2)          
    elif dkmt1==8:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]+a
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]+a
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]+a    
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]+a
        rho6=rho[z[x]-5,x]
        dz6=dz[z[x]-5,x]+a  
        rho7=rho[z[x]-6,x]
        dz7=dz[z[x]-6,x]+a
        rho8=rho[z[x]-7,x]
        dz8=dz[z[x]-7,x]+a        
        rho9=rho[z[x]-8,x]
        dz9=dz[z[x]-8,x]- dz[z[x]-8,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8+dz9)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8+dz9)
               -g*rho4*dz4*(dz5+dz6+dz7+dz8+dz9)
               -g*rho5*dz5*(dz6+dz7+dz8+dz9)
               -g*rho6*dz6*(dz7+dz8+dz9)
               -g*rho7*dz7*(dz8+dz9)
               -g*rho8*dz8*(dz9)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2-g*rho9*(dz9)**2/2)                 
    elif dkmt1==-8:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]+b
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]+b
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]+b    
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]+b
        rho6=rho[z[x-1]-5,x-1]
        dz6=dz[z[x-1]-5,x-1]+b
        rho7=rho[z[x-1]-6,x-1]
        dz7=dz[z[x-1]-6,x-1]+b    
        rho8=rho[z[x-1]-7,x-1]
        dz8=dz[z[x-1]-7,x-1]+b        
        rho9=rho[z[x-1]-8,x-1]
        dz9=dz[z[x-1]-8,x-1]- dz[z[x-1]-8,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8+dz9)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8+dz9)
               -g*rho4*dz4*(dz5+dz6+dz7+dz8+dz9)
               -g*rho5*dz5*(dz6+dz7+dz8+dz9)
               -g*rho6*dz6*(dz7+dz8+dz9)
               -g*rho7*dz7*(dz8+dz9)
               -g*rho8*dz8*(dz9)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2-g*rho9*(dz9)**2/2)         
    elif dkmt1==9:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]+a
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]+a
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x] +a   
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]+a
        rho6=rho[z[x]-5,x]
        dz6=dz[z[x]-5,x]+a
        rho7=rho[z[x]-6,x]
        dz7=dz[z[x]-6,x]+a
        rho8=rho[z[x]-7,x]
        dz8=dz[z[x]-7,x]+a
        rho9=rho[z[x]-8,x]
        dz9=dz[z[x]-8,x]+a        
        rho10=rho[z[x]-9,x]
        dz10=dz[z[x]-9,x]- dz[z[x]-9,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho4*dz4*(dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho5*dz5*(dz6+dz7+dz8+dz9+dz10)
               -g*rho6*dz6*(dz7+dz8+dz9+dz10)
               -g*rho7*dz7*(dz8+dz9+dz10)
               -g*rho8*dz8*(dz9+dz10)
               -g*rho9*dz9*(dz10)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2-g*rho9*(dz9)**2/2
               -g*rho10*(dz10)**2/2)                                   
    elif dkmt1==-9:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]+b
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]+b
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]+b    
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]+b
        rho6=rho[z[x-1]-5,x-1]
        dz6=dz[z[x-1]-5,x-1]+b
        rho7=rho[z[x-1]-6,x-1]
        dz7=dz[z[x-1]-6,x-1]+b 
        rho8=rho[z[x-1]-7,x-1]
        dz8=dz[z[x-1]-7,x-1]+b
        rho9=rho[z[x-1]-8,x-1]
        dz9=dz[z[x-1]-8,x-1]+b        
        rho10=rho[z[x-1]-9,x-1]
        dz10=dz[z[x-1]-9,x-1]- dz[z[x-1]-9,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho4*dz4*(dz5+dz6+dz7+dz8+dz9+dz10)
               -g*rho5*dz5*(dz6+dz7+dz8+dz9+dz10)
               -g*rho6*dz6*(dz7+dz8+dz9+dz10)
               -g*rho7*dz7*(dz8+dz9+dz10)
               -g*rho8*dz8*(dz9+dz10)
               -g*rho9*dz9*(dz10)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2-g*rho9*(dz9)**2/2
               -g*rho10*(dz10)**2/2)                                   
    elif dkmt1==-11:
        rho1=rho[z[x-1],x-1]
        dz1=dz[z[x-1],x-1]+b
        rho2=rho[z[x-1]-1,x-1]
        dz2=dz[z[x-1]-1,x-1]+b
        rho3=rho[z[x-1]-2,x-1]
        dz3=dz[z[x-1]-2,x-1]+b
        rho4=rho[z[x-1]-3,x-1]
        dz4=dz[z[x-1]-3,x-1]+b    
        rho5=rho[z[x-1]-4,x-1]
        dz5=dz[z[x-1]-4,x-1]+b
        rho6=rho[z[x-1]-5,x-1]
        dz6=dz[z[x-1]-5,x-1]+b
        rho7=rho[z[x-1]-6,x-1]
        dz7=dz[z[x-1]-6,x-1]+b 
        rho8=rho[z[x-1]-7,x-1]
        dz8=dz[z[x-1]-7,x-1]+b
        rho9=rho[z[x-1]-8,x-1]
        dz9=dz[z[x-1]-8,x-1]+b
        rho10=rho[z[x-1]-9,x-1]
        dz10=dz[z[x-1]-9,x-1]+b
        rho11=rho[z[x-1]-10,x-1]
        dz11=dz[z[x-1]-10,x-1]+b        
        rho12=rho[z[x-1]-11,x-1]
        dz12=dz[z[x-1]-11,x-1]- dz[z[x-1]-11,x]+b
        dp=dp-(pb[x-1]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12)
               -g*rho4*dz4*(dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12)
               -g*rho5*dz5*(dz6+dz7+dz8+dz9+dz10+dz11+dz12)
               -g*rho6*dz6*(dz7+dz8+dz9+dz10+dz11+dz12)
               -g*rho7*dz7*(dz8+dz9+dz10+dz11+dz12)
               -g*rho8*dz8*(dz9+dz10+dz11+dz12)
               -g*rho9*dz9*(dz10+dz11+dz12)
               -g*rho10*dz10*(dz11+dz12)
               -g*rho11*dz11*(dz12)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2-g*rho9*(dz9)**2/2
               -g*rho10*(dz10)**2/2-g*rho11*(dz11)**2/2-g*rho12*(dz12)**2/2)                                   
    elif dkmt1==12:
        rho1=rho[z[x],x]
        dz1=dz[z[x],x]+a
        rho2=rho[z[x]-1,x]
        dz2=dz[z[x]-1,x]+a
        rho3=rho[z[x]-2,x]
        dz3=dz[z[x]-2,x]+a
        rho4=rho[z[x]-3,x]
        dz4=dz[z[x]-3,x]+a    
        rho5=rho[z[x]-4,x]
        dz5=dz[z[x]-4,x]+a
        rho6=rho[z[x]-5,x]
        dz6=dz[z[x]-5,x]+a
        rho7=rho[z[x]-6,x]
        dz7=dz[z[x]-6,x]+a 
        rho8=rho[z[x]-7,x]
        dz8=dz[z[x]-7,x]+a
        rho9=rho[z[x]-8,x]
        dz9=dz[z[x]-8,x]+a
        rho10=rho[z[x]-9,x]
        dz10=dz[z[x]-9,x]+a
        rho11=rho[z[x]-10,x]
        dz11=dz[z[x]-10,x]+a
        rho12=rho[z[x]-11,x]
        dz12=dz[z[x]-11,x]+a
        rho13=rho[z[x]-12,x]        
        dz13=dz[z[x]-12,x]- dz[z[x]-12,x-1]+a
        dp=dp+(pb[x]*(dz1+dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12+dz13)
               -g*rho1*dz1*(dz2+dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12+dz13)
               -g*rho2*dz2*(dz3+dz4+dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12+dz13)
               -g*rho3*dz3*(dz4+dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12+dz13)
               -g*rho4*dz4*(dz5+dz6+dz7+dz8+dz9+dz10+dz11+dz12+dz13)
               -g*rho5*dz5*(dz6+dz7+dz8+dz9+dz10+dz11+dz12+dz13)
               -g*rho6*dz6*(dz7+dz8+dz9+dz10+dz11+dz12+dz13)
               -g*rho7*dz7*(dz8+dz9+dz10+dz11+dz12+dz13)
               -g*rho8*dz8*(dz9+dz10+dz11+dz12+dz13)
               -g*rho9*dz9*(dz10+dz11+dz12+dz13)
               -g*rho10*dz10*(dz11+dz12+dz13)
               -g*rho11*dz11*(dz12+dz13)
               -g*rho12*dz12*(dz13)
               -g*rho1*(dz1)**2/2-g*rho2*(dz2)**2/2-g*rho3*(dz3)**2/2
               -g*rho4*(dz4)**2/2-g*rho5*(dz5)**2/2-g*rho6*(dz6)**2/2
               -g*rho7*(dz7)**2/2-g*rho8*(dz8)**2/2-g*rho9*(dz9)**2/2
               -g*rho10*(dz10)**2/2-g*rho11*(dz11)**2/2-g*rho12*(dz12)**2/2
               -g*rho13*(dz13)**2/2)                                   
                                  

In [36]:
dp

<xarray.DataArray ()>
array(-2485855.410936117)
Coordinates:
    yt_ocean  float64 -57.23
    st_ocean  float64 1.683e+03

## Step 3: compare with wind stress at this latitude

In [37]:
taux.sum('xu_ocean')

<xarray.DataArray ()>
array(2682698.5165284337)
Coordinates:
    yu_ocean  float64 -57.16

## Summary:
### 1. Two latitudes were tested in this code.
### 2. (yt_ocean=217) give $\sum \Delta p \Delta z=-2.5*10^6$ at $\tau_{wind}=2.68*10^6$
###  (yt_ocean=220) give $\sum \Delta p \Delta z=-6*10^6$ at $\tau_{wind}=2.73*10^6$
### 3. This method may not suitable for fixing this problem, but it indicates that the fluctuations in dz due to surface wave will affect the calculation.

